In [103]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np
from google.colab import files

# RNN lyric generator:
We are going to use an RNN to show it a sample of a song we want it to recreate and let it learn to write a version of its own. This will be done using character predictive model.

The input will come in as a variable length sequence and were then going to predict the next character. This model can be used many times in a row with the output from the last prediction as the input for the nect call to generate a sequence.

# Dataset:
 We can write our own song and pass that to the network to learn. 

In [104]:
pathToFile=  list(files.upload().keys())[0]

Saving sickomode.txt to sickomode.txt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Reading File Contents

In [105]:
#Reading File
text = open(pathToFile,'rb').read().decode(encoding='utf-8')

#length of text is number of characters in it
print('Length of text:', len(text))

Length of text: 6661


In [106]:
# the first 250 characters in the text
print(text[:250])

Astro, yeah
Sun is down, freezin' cold
That's how we already know winter's here
My dawg would prolly do it for a Louis belt
That's just all he know, he don't know nothin' else
I tried to show 'em, yeah
I tried to show 'em, yeah, yeah
Yeah, yeah, yeah


# Encoding
preprocessing step to encode each unique character as a different integer

In [107]:
#gets the vocabulary of file
vocab= sorted(set(text))

#Create mapping from unique characters to indices
char2ind = {u:i for i, u in enumerate(vocab)} 
ind2char = np.array(vocab)

def textToInt(text):
  return np.array([char2ind[c] for c in text])

textAsInts= textToInt(text)
#testing fcn
print('Text:', text[:13])
print('Encoded:', textToInt(text[:13]))

Text: Astro, yeah
S
Encoded: [16 57 58 56 54  7  1 63 44 40 47  0 33]


In [108]:
#converts encoded integers back to their respective strings
def intToText(ints):
  try:
    ints= ints.numpy()
  except:
    pass
  return ''.join(ind2char[ints])

print(intToText(textAsInts[:13]))

Astro, yeah
S


Creating Training Examples

need to split our data from above into many shorter sequences to pass to the model as training examples

In [109]:
seqlength= 100
examplesPerEpoch = len(text)//(seqlength+1)

#creating training examples / targets
charDataset= tf.data.Dataset.from_tensor_slices(textAsInts)

In [110]:
## use batch method to turn stream of characters into batches of desried length
sequences= charDataset.batch(seqlength+1, drop_remainder=True)

In [111]:
## use these sequence lengths of 101 and split them into input and output

def splitInputTarget(chunk):
  input= chunk[:-1] #hell
  target= chunk[1:] #ello
  return input,target 

dataset= sequences.map(splitInputTarget) #we use map to apply the above fcn to every entry

Creating Training Batches

In [112]:
BATCH_SIZE = 64
VOCAB_SIZE= len(vocab)
EMBEDDING_DIM= 256
RNN_UNITS= 1024

## buffer size to shuffle dataset 
BUFFER_SIZE= 10000
data= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder= True)


# Building the Model 
well use an embedding layer, an lstm and a dense layer that contains a node for each character in our training data. The dense layer is gonna give us a probability distribution over all the nodes

In [113]:
def buildModel(vocabSize, embeddingDim,rnnUnits,batchSize):
  model = tf.keras.Sequential([
          tf.keras.layers.Embedding(vocabSize, embeddingDim,batch_input_shape=[batchSize,None]),
          tf.keras.layers.LSTM(
              rnnUnits,
              return_sequences=True,
              stateful=True,
              recurrent_initializer= 'glorot_uniform'),
          tf.keras.layers.Dense(vocabSize) 
                               ])
  return model

model = buildModel(VOCAB_SIZE,EMBEDDING_DIM,RNN_UNITS,BATCH_SIZE)
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (64, None, 256)           16896     
_________________________________________________________________
lstm_5 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_5 (Dense)              (64, None, 66)            67650     
Total params: 5,331,522
Trainable params: 5,331,522
Non-trainable params: 0
_________________________________________________________________


# Creating a loss function:
This is because our model will out a (64,seqlen, 65) shaped tensor that represents the probability distributiom of each character at each timestep for every sequence in the batch

In [114]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions= model(input_example_batch)
  print(example_batch_predictions.shape, '(batchsize, seqlen vocabsize')

(64, 100, 66) (batchsize, seqlen vocabsize


In [115]:
# prediction is an array of 64 arrays , one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-1.46075431e-03  6.95515890e-04  1.37151661e-03 ...  1.12645945e-03
    1.02751597e-03 -3.89817986e-03]
  [ 6.07406814e-03 -4.52859327e-04  8.03739706e-04 ...  4.56535537e-03
    4.71199770e-03 -4.25138511e-03]
  [ 8.12957250e-03  1.98156387e-03 -4.16179840e-03 ...  9.94851580e-04
   -1.52764213e-03 -8.71043373e-03]
  ...
  [-5.60639007e-03  3.55044147e-03  4.85069398e-03 ...  7.44607020e-03
    4.28133039e-03  5.28511778e-03]
  [-5.14288526e-03  4.11923090e-03  6.01298595e-03 ...  6.44449983e-03
    4.39730147e-03  1.18047779e-03]
  [-4.76413965e-03  7.11829728e-03  9.77694150e-03 ...  1.08335000e-02
    5.48578659e-03  5.71348565e-03]]

 [[-2.31600739e-03  3.33094643e-03 -8.82234744e-05 ...  1.92274898e-03
    1.30965724e-03  2.40115356e-03]
  [-3.84219363e-03  6.14783121e-03  4.21063369e-03 ...  8.05145595e-03
    2.29165284e-03  5.91642829e-03]
  [-2.36135116e-03  1.44351204e-03  2.59647542e-03 ...  6.84021180e-03
    3.58058093e-03  9.74581752e-04]
  ...
  [ 1.655

In [116]:
#example of one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[-0.00146075  0.00069552  0.00137152 ...  0.00112646  0.00102752
  -0.00389818]
 [ 0.00607407 -0.00045286  0.00080374 ...  0.00456536  0.004712
  -0.00425139]
 [ 0.00812957  0.00198156 -0.0041618  ...  0.00099485 -0.00152764
  -0.00871043]
 ...
 [-0.00560639  0.00355044  0.00485069 ...  0.00744607  0.00428133
   0.00528512]
 [-0.00514289  0.00411923  0.00601299 ...  0.0064445   0.0043973
   0.00118048]
 [-0.00476414  0.0071183   0.00977694 ...  0.0108335   0.00548579
   0.00571349]], shape=(100, 66), dtype=float32)


In [117]:
# looking at prediction at the first timestep
time_pred= pred[0]
print(len(time_pred))
print(time_pred) # 65 values representing the probability of each character occuring next

66
tf.Tensor(
[-1.4607543e-03  6.9551589e-04  1.3715166e-03  4.3434398e-03
  2.2906051e-03 -3.7867003e-03 -3.4636247e-03  7.4635725e-04
 -9.8098314e-04  4.0042223e-03  1.4077584e-03 -3.5756957e-03
  6.8338132e-03  2.6984089e-03 -1.0621442e-03  2.5670924e-03
  1.1328957e-05 -6.9139078e-03  2.0805574e-03 -6.8662246e-04
  4.9338462e-03  8.7010786e-03 -4.6818057e-04 -1.0417296e-03
 -1.6846646e-03 -2.5506911e-03  1.2777384e-03 -1.0323012e-03
  2.2772073e-03 -5.3225607e-03 -1.9739303e-03  7.3367930e-03
  2.6972985e-03  3.2319548e-04 -2.9315052e-03  2.1871948e-03
  1.2479043e-03  1.1634380e-03 -2.2547971e-03 -1.7501600e-03
  4.8372708e-03  2.2759666e-03  2.4410523e-03  1.5703047e-03
 -3.1239481e-03  1.6869872e-03 -1.1587087e-03 -4.9082614e-03
 -6.5459735e-03 -4.5301360e-03  1.0555290e-03  2.1599710e-04
 -2.3797927e-03  5.5100775e-04  6.9787642e-03  2.7469122e-03
  3.2007438e-04  5.5280863e-04  2.1205470e-04  1.3130674e-03
 -2.2845422e-03 -4.1078805e-04  6.6338764e-03  1.1264595e-03
  1.027516

In [118]:
# to determine the next in sequence we sample output distributiong (pick a value based on probability)
sampledIndices= tf.random.categorical(pred,num_samples=1)

#reshape the array and convert  all integers to see actual characters
sampledIndices= np.reshape(sampledIndices, (1,-1))[0]
predictedChars= intToText(sampledIndices)

predictedChars ## this is what the model predicted for training sequence 1

'J\nF,L3yCPW,FHCckvoP7Hi(w\nYrje6UGsslévPKBVAUt)hYiCnm\nvitvMJPnXSvy,n\'W)tu)E3"éabcsvvX?T660lé7nkvY-0éT-'

We create a loss function that compares the output to the expected output and give us some numeric value representing how close the two were

In [119]:
def loss(labels,logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels,logits,from_logits=True)

# Compiling the Model
Model predicts the probability of each unique letter coming next

In [120]:
model.compile(optimizer='adam', loss=loss)

# Checkpoint Creation
Configuring our model to save and checkpoints as it trains

In [121]:
#Directory to where the checkpoints will be saved
checkpointDir= ',/trainingCheckpoints'

#name of checkpoint files
checkpointPrefix= os.path.join(checkpointDir, 'ckpt_{epoch}')
checkpointCallback= tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpointPrefix,
    save_weights_only=True)

# Training

In [122]:
history= model.fit(data,epochs=600, callbacks=[checkpointCallback])

Epoch 1/600
1/1 [==============================] - 1s 1s/step - loss: 4.1890
Epoch 2/600
1/1 [==============================] - 0s 118ms/step - loss: 4.1406
Epoch 3/600
1/1 [==============================] - 0s 120ms/step - loss: 3.7038
Epoch 4/600
1/1 [==============================] - 0s 101ms/step - loss: 3.8112
Epoch 5/600
1/1 [==============================] - 0s 97ms/step - loss: 3.6840
Epoch 6/600
1/1 [==============================] - 0s 111ms/step - loss: 3.5146
Epoch 7/600
1/1 [==============================] - 0s 113ms/step - loss: 3.2904
Epoch 8/600
1/1 [==============================] - 0s 117ms/step - loss: 3.2440
Epoch 9/600
1/1 [==============================] - 0s 119ms/step - loss: 3.2849
Epoch 10/600
1/1 [==============================] - 0s 108ms/step - loss: 3.1966
Epoch 11/600
1/1 [==============================] - 0s 101ms/step - loss: 3.1908
Epoch 12/600
1/1 [==============================] - 0s 94ms/step - loss: 3.2086
Epoch 13/600
1/1 [========================

# Loading the Model
Well rebuild the model from a checkpoint using a batch size of 1 so that we can feed one piece of text to the model and have it make a prediction

In [123]:
model= buildModel(VOCAB_SIZE,EMBEDDING_DIM,RNN_UNITS,1)

Once the model is finished training we can find the LASTEST checkpoint that stores the models weigths using the following line 

In [124]:
model.load_weights(tf.train.latest_checkpoint(checkpointDir))
model.build(tf.TensorShape([1,None]))

We can load any checkpoint we want by specifying the exact file to load

In [125]:
#checkpointNum= 10
#model.load_weights(tf.train.load_checkpoint('./trainingCheckpoints/ckpt' + str(checkpointNum)))

# Generating Text


In [126]:
def generate_text(model,start_string):
  #Evaluation step(generating text using the learned model)

  #num of characters to generate
  numGenerate= 800

  #converting our start string to numbers {vectorizing}
  inputEval= [char2ind[s] for s in start_string]
  inputEval=  tf.expand_dims(inputEval,0)

  #Empty string to store our results 
  textGenerated= []

  ##low temps results in more predictable texts
  # higher temperatures result in more surprising text
  # Experiment to find best setting
  temperature= 1.0

  #here batch size == 1
  model.reset_states()
  for i in range(numGenerate):
    predictions = model(inputEval)
    #remove batch dimension
    predictions= tf.squeeze(predictions,0)

    #using categorical distribution to predict the character returned by the model
    predictions= predictions/temperature
    predicted_id= tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()

    ## we pass the predicted charcter as  the nect input to the model along w the previous hidden state

    inputEval= tf.expand_dims([predicted_id],0)

    textGenerated.append(ind2char[predicted_id])

  return (start_string + ''.join(textGenerated))


enjoy

In [129]:
inp= input('Type a starting string: ')
print('\n', generate_text(model,inp))

Type a starting string: Young Pharoah

 Young Pharoah
we ondir you how a light show (Ooh)
At the night show (Ooh)
Your bitch not at home, she at the night show (Ooh)
Fuckin' rin high school, I used to bus it to the dance (yeah)
Now I hit the FBO with duffles in my hands
I dido it for a Louis belt
That's just all he know, he don't know nothin' else
I tried to show 'em, yeah
At the night show
At the night show
Everything can happen at the night show (Ooh)
At the night show (Ooh)
At the night show (Ooh)
Your bitch not at home, she at the night show (Ooh)
Fuckin' rin hizg sol woen to show the you nttook (yeah)
Bat man't to no dy it the ght show (Ooh)
At the night show (Ooh)
Your bitch not at home, she at the night show (Ooh)
Fuckin' rin high school, I used to bus it to the dance (yeah)
Now I hit the FBO with duffles in my hands
I did the dawgs a
